In [1]:
from IPython.display import display
from one_locus_two_alleles_app import get_app
app_name_1 = "allelic_freqs_hw_assumed_app"
app_name_2 = "simple_one_pop_app"
app_name_3 = "full_one_pop_app"
display(get_app(app_name_1))

OneLociTwoAllelesSimulationApp(children=(HBox(children=(Label(value='Freq. A'), FloatSlider(value=0.5, max=1.0…